## Run Sentence Transformer Model Without Fine-Tuning

In [2]:
import json
import os
import sys
import lancedb
import numpy as np
from sentence_transformers.cross_encoder import CrossEncoder

# Add the week1_bootstrap_evals to path to import scoring_utils
sys.path.append(os.path.abspath("../week1_bootstrap_evals"))
from scoring_utils import EvalQuestion

model = CrossEncoder("cross-encoder/stsb-distilroberta-base")
with open("../week1_bootstrap_evals/synthetic_eval_dataset.json", "r") as f:
    synthetic_questions = json.load(f)
eval_questions = [EvalQuestion(**q) for q in synthetic_questions]

db = lancedb.connect("../week1_bootstrap_evals/lancedb")
reviews_table = db.open_table("reviews")

def score_question(eval_question: EvalQuestion):
    query = f"Answer the following question: {eval_question.question_with_context}\n."
    target_id = int(eval_question.chunk_id)
    first_stage = reviews_table.search(query).limit(50).to_pandas()
    first_stage_ids = first_stage.id.astype(int).values
    review_text = first_stage.review.values
    reranked_results = model.rank(query, review_text)
    is_right_result = lambda x: first_stage_ids[x["corpus_id"]] == target_id
    try:
        rank_of_desired_result = next(
            i for i, d in enumerate(reranked_results) if 
            is_right_result(d)
        )
    except StopIteration:
        return np.inf
    return rank_of_desired_result

ranks = [score_question(eval_question) for eval_question in eval_questions]
recall_at_5 = np.mean([rank <= 5 for rank in ranks])
recall_at_10 = np.mean([rank <= 10 for rank in ranks])
recall_at_20 = np.mean([rank <= 20 for rank in ranks])
print(f"Recall at 5: {recall_at_5}\nRecall at 10: {recall_at_10}\nRecall at 20: {recall_at_20}")

/Users/dan/Desktop/systematically-improving-rag/systemic/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Recall at 5: 0.6044444444444445
Recall at 10: 0.92
Recall at 20: 0.9811111111111112


(0.6044444444444445, 0.92, 0.9811111111111112)